In [3]:
import urllib.request
from bs4 import BeautifulSoup
from multiprocessing.dummy import Pool as ThreadPool
import os

'''
find all zip files from a page and download them all
target webpage should be FTP like, meaning, all downloadable links are enclosed in href tag

For example:
https://www2.census.gov/geo/tiger/TIGER2021/BG/
https://downloads-direct.freemdict.com/Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/A1%E5%90%AC%E5%8A%9B/
'''

archive_url="https://downloads-direct.freemdict.com/Language_Learning_Videos/French/004%20%E4%BD%A0%E5%A5%BD%E6%B3%95%E8%AF%AD%200-B1/A2%E5%90%AC%E5%8A%9B/"
link = urllib.request.urlopen(archive_url)
r = link.read()
soup = BeautifulSoup(r, 'html.parser')

# Create two lists, one for links one for filenames
# Used to create link-name paires used in Pool
# Pool is used so we can download multifiles at the sametime
target_files=[]
filenames=[]
for a in soup.find_all('a', href=True):
    if str(a['href']).endswith(".mp3"):
        file_name=str(a['href'])
        file_url=archive_url+file_name
        target_files.append(file_url)
        filenames.append(file_name)

if not os.path.exists(".\\mp3\\"):
    os.os.mkdir(".\\mp3\\")
def get_file(link,file_name):
    urllib.request.urlretrieve(link, f".\\mp3\\{file_name}")

pool = ThreadPool(5)
pool.starmap(get_file, zip(target_files,filenames))

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]